In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


# RAG 성능 테스트를 위한 함수 정의

In [2]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

# PDFRAG 객체 생성
rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
)

# 검색기(retriever) 생성
retriever = rag.create_retriever()

# 체인(chain) 생성
chain = rag.create_chain(retriever)

# 질문에 대한 답변 생성
chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

In [3]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

# 한글 형태소 분석기의 활용

In [4]:
from langchain_teddynote.community.kiwi_tokenizer import KiwiTokenizer

# 토크나이저 선언
kiwi_tokenizer = KiwiTokenizer()

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"

# 토큰화
print(sent1.split())
print(sent2.split())

print("===" * 20)

# 토큰화
print(kiwi_tokenizer.tokenize(sent1))
print(kiwi_tokenizer.tokenize(sent2))

['안녕하세요.', '반갑습니다.', '내', '이름은', '테디입니다.']
['안녕하세용', '반갑습니다~^^', '내', '이름은', '테디입니다!!']
['안녕', '하', '세요', '.', '반갑', '습니다', '.', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '.']
['안녕', '하', '세요', 'ᆼ', '반갑', '습니다', '~', '^^', '나', '의', '이름', '은', '테디', '이', 'ᆸ니다', '!!']


# Rouge (Recall-Oriented Understudy for Gisting Evaluation) 스코어

In [6]:
from rouge_score import rouge_scorer

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], use_stemmer=False, tokenizer=KiwiTokenizer()
)

print(
    f"[1] {sent1}\n[2] {sent2}\n[rouge1] {scorer.score(sent1, sent2)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent2)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent2)['rougeL'].fmeasure:.5f}"
)
print("===" * 20)
print(
    f"[1] {sent1}\n[2] {sent3}\n[rouge1] {scorer.score(sent1, sent3)['rouge1'].fmeasure:.5f}\n[rouge2] {scorer.score(sent1, sent3)['rouge2'].fmeasure:.5f}\n[rougeL] {scorer.score(sent1, sent3)['rougeL'].fmeasure:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[rouge1] 0.77419
[rouge2] 0.62069
[rougeL] 0.77419
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[rouge1] 1.00000
[rouge2] 0.92857
[rougeL] 0.53333


# BLEU (Bilingual Evaluation Understudy) 스코어

In [7]:
from nltk.translate.bleu_score import sentence_bleu

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# 토큰화
print(kiwi_tokenizer.tokenize(sent1, type="sentence"))
print(kiwi_tokenizer.tokenize(sent2, type="sentence"))
print(kiwi_tokenizer.tokenize(sent3, type="sentence"))

안녕 하 세요 . 반갑 습니다 . 나 의 이름 은 테디 이 ᆸ니다 .
안녕 하 세요 ᆼ 반갑 습니다 ~ ^^ 나 의 이름 은 테디 이 ᆸ니다 !!
나 의 이름 은 테디 이 ᆸ니다 . 안녕 하 세요 . 반갑 습니다 .


In [8]:
bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent2, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent2}\n[score] {bleu_score:.5f}")
print("===" * 20)

bleu_score = sentence_bleu(
    [kiwi_tokenizer.tokenize(sent1, type="sentence")],
    kiwi_tokenizer.tokenize(sent3, type="sentence"),
)
print(f"[1] {sent1}\n[2] {sent3}\n[score] {bleu_score:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.74879
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.95739


# SemScore

In [10]:
from sentence_transformers import SentenceTransformer, util
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

sent1 = "안녕하세요. 반갑습니다. 내 이름은 테디입니다."
sent2 = "안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!"
sent3 = "내 이름은 테디입니다. 안녕하세요. 반갑습니다."

# SentenceTransformer 모델 로드
model = SentenceTransformer("all-mpnet-base-v2")

# 문장들을 인코딩
sent1_encoded = model.encode(sent1, convert_to_tensor=True)
sent2_encoded = model.encode(sent2, convert_to_tensor=True)
sent3_encoded = model.encode(sent3, convert_to_tensor=True)

# sent1과 sent2 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent2_encoded).item()
print(f"[1] {sent1}\n[2] {sent2}\n[score] {cosine_similarity:.5f}")

print("===" * 20)

# sent1과 sent3 사이의 코사인 유사도 계산
cosine_similarity = util.pytorch_cos_sim(sent1_encoded, sent3_encoded).item()
print(f"[1] {sent1}\n[2] {sent3}\n[score] {cosine_similarity:.5f}")

[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 안녕하세용 반갑습니다~^^ 내 이름은 테디입니다!!
[score] 0.86157
[1] 안녕하세요. 반갑습니다. 내 이름은 테디입니다.
[2] 내 이름은 테디입니다. 안녕하세요. 반갑습니다.
[score] 0.99191


In [11]:
from langsmith.schemas import Run, Example
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from sentence_transformers import SentenceTransformer, util
import os

# 토크나이저 병렬화 설정(HuggingFace 모델 사용)
os.environ["TOKENIZERS_PARALLELISM"] = "true"


def rouge_evaluator(metric: str = "rouge1") -> dict:
    # wrapper function 정의
    def _rouge_evaluator(run: Run, example: Example) -> dict:
        # 출력값과 정답 가져오기
        student_answer = run.outputs.get("answer", "")
        reference_answer = example.outputs.get("answer", "")

        # ROUGE 점수 계산
        scorer = rouge_scorer.RougeScorer(
            ["rouge1", "rouge2", "rougeL"], use_stemmer=True, tokenizer=KiwiTokenizer()
        )
        scores = scorer.score(reference_answer, student_answer)

        # ROUGE 점수 반환
        rouge = scores[metric].fmeasure

        return {"key": "ROUGE", "score": rouge}

    return _rouge_evaluator


def bleu_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="sentence")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="sentence")

    # BLEU 점수 계산
    bleu_score = sentence_bleu([reference_tokens], student_tokens)

    return {"key": "BLEU", "score": bleu_score}


def meteor_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # 토큰화
    reference_tokens = kiwi_tokenizer.tokenize(reference_answer, type="list")
    student_tokens = kiwi_tokenizer.tokenize(student_answer, type="list")

    # METEOR 점수 계산
    meteor = meteor_score.meteor_score([reference_tokens], student_tokens)

    return {"key": "METEOR", "score": meteor}


def semscore_evaluator(run: Run, example: Example) -> dict:
    # 출력값과 정답 가져오기
    student_answer = run.outputs.get("answer", "")
    reference_answer = example.outputs.get("answer", "")

    # SentenceTransformer 모델 로드
    model = SentenceTransformer("all-mpnet-base-v2")

    # 문장 임베딩 생성
    student_embedding = model.encode(student_answer, convert_to_tensor=True)
    reference_embedding = model.encode(reference_answer, convert_to_tensor=True)

    # 코사인 유사도 계산
    cosine_similarity = util.pytorch_cos_sim(
        student_embedding, reference_embedding
    ).item()

    return {"key": "sem_score", "score": cosine_similarity}


In [12]:
from langsmith.evaluation import evaluate

# 평가자 정의
heuristic_evalulators = [
    rouge_evaluator(metric="rougeL"),
    bleu_evaluator,
    meteor_evaluator,
    semscore_evaluator,
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실험 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=heuristic_evalulators,
    experiment_prefix="Heuristic-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Heuristic-EVAL (Rouge, BLEU, METEOR, SemScore) 을 사용하여 평가",
    },
)


View the evaluation results for experiment: 'Heuristic-EVAL-c903d355' at:
https://smith.langchain.com/o/9b141874-d093-4103-946d-7bc247255f98/datasets/899fb1c5-744d-4f35-a48e-68fe78d807f1/compare?selectedSessions=14722654-96be-4539-852f-a2ac67c91894




0it [00:00, ?it/s]

Error running evaluator <DynamicRunEvaluator meteor_evaluator> on run 6045be35-256b-4345-8188-0bbd25b2ea09: LookupError("\n**********************************************************************\n  Resource \x1bwordnet\x1b not found.\n  Please use the NLTK Downloader to obtain the resource:\n\n  \x1b>>> import nltk\n  >>> nltk.download('wordnet')\n  \x1b\n  For more information see: https://www.nltk.org/data.html\n\n  Attempted to load \x1bcorpora/wordnet\x1b\n\n  Searched in:\n    - 'C:\\\\Users\\\\kakao/nltk_data'\n    - 'C:\\\\Users\\\\kakao\\\\AI_study\\\\.venv\\\\nltk_data'\n    - 'C:\\\\Users\\\\kakao\\\\AI_study\\\\.venv\\\\share\\\\nltk_data'\n    - 'C:\\\\Users\\\\kakao\\\\AI_study\\\\.venv\\\\lib\\\\nltk_data'\n    - 'C:\\\\Users\\\\kakao\\\\AppData\\\\Roaming\\\\nltk_data'\n    - 'C:\\\\nltk_data'\n    - 'D:\\\\nltk_data'\n    - 'E:\\\\nltk_data'\n**********************************************************************\n")
Traceback (most recent call last):
  File "C:\Users\kak